In [1]:
pip install petl

     -------------------------------------- 411.7/411.7 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for petl: filename=petl-1.7.14-py3-none-any.whl size=229771 sha256=dd901f19e3fedde4a8e2b3032faf9a9bec1b7ca44075c4188154492f88eef816
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\71\0b\8b\aaa862db0638ab059cacc3d1b33edacb853e88285f9f7cb3d4
Successfully built petl
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymssql

     ---------------------------------------- 2.0/2.0 MB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [76]:
# importing libraries
import os
import petl
import pymssql
import configparser
import requests
import datetime
import json
import decimal
import sys


EXTRACT

In [58]:
# get data from config File
config = configparser.ConfigParser()
try:
    config.read('config.ini')
except Exception as e:
    print("Could not read Configuration File" + str(e))
    sys.exit()

In [59]:
# read settings from configuration file
startdate = config['CONFIG']['startdate']
url = config['CONFIG']['url']
destServer = config['CONFIG']['server']
destDatabase = config['CONFIG']['database']

In [60]:
# request data form url
try:
    BOCResponse = requests.get(url+startdate)
except Exception as e:
    print("Could not make request"+ str(e))
    sys.exit()

In [61]:
# initialize list of lists for data storage

BOCDates = []
BOCRates = []

TRANSFORM

In [62]:
# check response status and process BOC JSON object
if (BOCResponse.status_code == 200):
    BOCRaw = json.loads(BOCResponse.text)

    # extract observation data in column array
    for row in BOCRaw['observations']:
        BOCDates.append(datetime.datetime.strptime(row['d'], "%Y-%m-%d"))
        BOCRates.append(decimal.Decimal(row['FXUSDCAD']['v']))

    # create a petl table form column array and rename to columns
    exchange_rate = petl.fromcolumns([BOCDates, BOCRates], header= ['date', 'rate'])
    
    # load expense date
    try:
        expenses = petl.io.xlsx.fromxlsx('Expenses.xlsx', sheet = 'Github')
    except Exception as e:
        print('Could not load data'+ str(e))
        sys.exit()
    
    # join tables
    expenses = petl.outerjoin(exchange_rate, expenses, key = "date")
    
    # filling missing values in expenses
    expenses = petl.filldown(expenses, 'rate')
    
    # remove dates with no expenses
    expenses = petl.select(expenses, lambda rec: rec.USD != None)
    
    # add CAD column
    expenses = petl.addfield(expenses, 'CAD', lambda rec: decimal.Decimal(rec.USD)*rec.rate)
    

LOAD

In [ ]:
# initialize database connection
try:
    dbconnection = pymssql.connect(server = destServer, database = destDatabase)
except Exception as e:
    print('Could not connect with database:'+str(e))
    sys.exit()


In [ ]:
#populate expenses database tables
try:
    petl.io.todb(expenses, dbconnection, 'Expenses')
except Exception as e:
    print('Could not populate table'+str(e))
    